In [ ]:
import pandas as pd
import sys
# print(sys.path)
# Adding a path so that it finds the package for numpy
sys.path.append('/usr/local/lib/python3.7/site-packages')
import numpy as np
import matplotlib.pyplot as plt
from random import randint
from collections import Counter
import seaborn as sns
from matplotlib.colors import ListedColormap
from matplotlib import cm

import chart_studio.plotly as py
import cufflinks as cf
import pandas as pd
%matplotlib inline
from ipywidgets import interact
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

cf.go_offline()

from bokeh.io import output_file, show
from bokeh.models import CheckboxGroup

In [ ]:
'The 2002–2004 SARS outbreak was an epidemic involving severe acute respiratory syndrome (SARS) caused by SARS-CoV' 
'The outbreak was first identified in Foshan, Guangdong, China, in November 2002.'
'Over 8,000 people from 29 different countries and territories were infected, and at least 774 died worldwide.'
'The World Health Organization declared severe acute respiratory syndrome contained on 5 July 2003'
'However several SARS cases were reported until May 2004.'
'Hong Kong had 1755 cases and 299 deaths (17% fatality rate)'
'Hong Konng has today around 1000 cases of COVID 19 and 4 deaths'

In [ ]:
'What was the impact of SARS on Retail and Horeca Sales?'
'What categories suffered the most?'
'How fast did the recovery took place?'
"What categories recovered first?"

In [ ]:
'First set of data includes Retail Sales by Category by month since 2001'
retail = pd.read_excel('../data/Retail Clean.xlsx')
ret = retail.describe()
ret.transpose()

In [ ]:
'Check that all year rows are clean and included'
retail['Year'].unique()

In [ ]:
' Create a function that eliminate 2001 and 2020 as they have partial data only. I will re-use it for the other dataframes'

def clean_year(df):
    return df[df['Year'].isin(['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'])]

In [ ]:
retail = clean_year(retail)

In [ ]:
'Create a date column by concatenating Year and Month. Date will be used for the plotting'
retail['Date'] = pd.to_datetime(retail['Month'].astype(str) + '/' + retail['Year'].astype(str))
retail.head()

In [ ]:
'Start by looking at the trend by entire year so to eliminate the impact of seasonality in the month by month'

def plot_bar(df, title, col_name):
    plt.rcParams['figure.figsize'] = [18.0, 6.0]
    sum_year = df[[col_name,'Year']].groupby(['Year']).agg("sum")
    sum_year.plot.bar()
    plt.title(title)
    return plt.show()

In [ ]:
plot_bar(retail,"HK Retail Sales - HK $","All retail outlets")

In [ ]:
'As expected, sales declined in 2003 vs 2002 and bounced back already in 2004.'

In [ ]:
'Now let''s go deeper and focus on the 2002 to 2005 period for a closer look'

retail_02_to_05 = retail[retail['Year'].isin(['2002','2003','2004','2005'])]

In [ ]:
'Look at the trend by month in the 2002 to 2005 range'

def plot_line(df, col_name, title):
    
    chart = sns.lineplot(df.Date, df[col_name])
    chart.set(xticks=df.Date.values)

    plt.xticks(
        rotation=45, 
        horizontalalignment='right',
        fontweight='light',
        fontsize='x-large'  
    )

    plt.title(title)
    return plt.show()


In [ ]:
plot_line(retail_02_to_05, 'All retail outlets', "Sales of Retail Outlets in Hong Kong ('000 HK$) - 2002-2005")

In [ ]:
'The fall in Retail sales takes place in M2 and continues to M4. A slow recover in Q3-Q4 and then a jum in M12 and M1 2004'

In [ ]:
'To get a better view of the impact, let''s look at the sales indexes of 2003 vs 2002'

In [ ]:
df = retail
    
df_2002 = df[df['Year'].isin(['2002'])]
df_2003 = df[df['Year'].isin(['2003'])]
    
Month = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
Flat = [100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100, 100]
    
data = {'Month': Month, 'Flat': Flat}
indf = pd.DataFrame (data)

col_names = [name for name in df.columns]
col_names.remove('Year')
col_names.remove('Month')
col_names.remove('Date')

for col_name in col_names:
        
    index_03vs02 = [int(tre/due*100) for tre,due in zip(df_2003[col_name],df_2002[col_name])]
    indf[col_name] = index_03vs02


In [ ]:
'Let us also calculate the speed of recovery as # of months it took for the category to go back to 100'

In [ ]:
speed = {}

indf_feb = indf.iloc[1:]

for col_name in col_names:
    counter = 0
    for index in indf_feb[col_name]:
        counter+=1
        if index > 100:
            break
        
    speed.update({col_name: counter}) 

In [ ]:
'And plot the two information'

In [ ]:
@interact
def Selection(Selection=col_names):
       
    fig, ax = plt.subplots()
    
    ax.plot(indf['Month'],indf[Selection], label=Selection, color="blue")
    ax.plot(indf['Month'],indf["Flat"], label='Flat', color="black", linewidth=0.5)

    ax.text('Dec', 100, speed[Selection], fontsize=30, color="blue")
    ax.legend(loc=4) # upper left corner
    ax.set_xlabel('Month', fontsize=18)
    ax.set_ylabel(Selection, fontsize=14)
    ax.set_title("2003 Sales vs 2002 same month & speed of recovery in # of months");

    return plt.show()


In [ ]:
'Total Retail sales recoevered in 7 months'
'Fresh fruits & vegetables were slow with 11 months'
'Supermarkets & Medicines the fastest with 2'


In [ ]:
'What categories were most impacted?'

In [ ]:
'Focus on the 2002 and 2003 years to make a comparison'

retail_02_and_03 = retail[retail['Year'].isin(['2002','2003'])]
retail_02_and_03_not_all = retail_02_and_03.drop(columns=['All retail outlets'])
sum_year__02_and_03_not_all = retail_02_and_03_not_all[['Fish, livestock and poultry, fresh or frozen',
                                     'Fruits and vegetables, fresh',
                                     'Bread, pastry, confectionery and biscuits',
                                     'Other food, not elsewhere classified',
                                     'Alcoholic drinks and tobacco',
                                     'Supermarkets',
                                     'Fuels',
                                     'Wearing apparel',
                                     'Footwear, allied products and other clothing accessories',
                                     'Motor vehicles and parts',
                                     'Furniture and fixtures',
                                     'Electrical goods and other consumer durable goods, not elsewhere classified',
                                     'Department stores',
                                     'Jewellery, watches and clocks, and valuable gifts',
                                     'Books, newspapers, stationery and gifts',
                                     'Chinese drugs and herbs',
                                     'Optical shops',
                                     'Medicines and cosmetics',
                                     'Other consumer goods, not elsewhere classified','Year']].groupby(['Year']).agg("sum")

In [ ]:
fig, ax = plt.subplots(figsize = (24,12))

size = 0.3

cmap = plt.get_cmap("tab20c")
outer_colors = sns.color_palette("GnBu", 20)
inner_colors = sns.color_palette("GnBu", 20)

ax.pie(sum_year__02_and_03_not_all.iloc[1], 
       radius=1,
       labels=sum_year__02_and_03_not_all.columns, 
       wedgeprops=dict(width=size, edgecolor='w'),
       colors=outer_colors,
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.pie(sum_year__02_and_03_not_all.iloc[0], 
       radius=1-size, 
       colors=inner_colors,
       wedgeprops=dict(width=size, edgecolor='w'),
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.set(aspect="equal", title='2003 (outer circle) vs 2002 (inner circle) Retail Sales by Category')
plt.show()

In [ ]:
'Focus on the 2003 and 2004 years to make a comparison'

retail_03_and_04 = retail[retail['Year'].isin(['2003','2004'])]
retail_03_and_04_not_all = retail_03_and_04.drop(columns=['All retail outlets'])
sum_year__03_and_04_not_all = retail_03_and_04_not_all[['Fish, livestock and poultry, fresh or frozen',
                                     'Fruits and vegetables, fresh',
                                     'Bread, pastry, confectionery and biscuits',
                                     'Other food, not elsewhere classified',
                                     'Alcoholic drinks and tobacco',
                                     'Supermarkets',
                                     'Fuels',
                                     'Wearing apparel',
                                     'Footwear, allied products and other clothing accessories',
                                     'Motor vehicles and parts',
                                     'Furniture and fixtures',
                                     'Electrical goods and other consumer durable goods, not elsewhere classified',
                                     'Department stores',
                                     'Jewellery, watches and clocks, and valuable gifts',
                                     'Books, newspapers, stationery and gifts',
                                     'Chinese drugs and herbs',
                                     'Optical shops',
                                     'Medicines and cosmetics',
                                     'Other consumer goods, not elsewhere classified','Year']].groupby(['Year']).agg("sum")

In [ ]:
fig, ax = plt.subplots(figsize = (24,12))

size = 0.3

cmap = plt.get_cmap("tab20c")
outer_colors = sns.color_palette("GnBu", 20)
inner_colors = sns.color_palette("GnBu", 20)

ax.pie(sum_year__03_and_04_not_all.iloc[1], 
       radius=1,
       labels=sum_year__03_and_04_not_all.columns, 
       wedgeprops=dict(width=size, edgecolor='w'),
       colors=outer_colors,
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.pie(sum_year__03_and_04_not_all.iloc[0], 
       radius=1-size, 
       colors=inner_colors,
       wedgeprops=dict(width=size, edgecolor='w'),
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.set(aspect="equal", title='2004 (outer circle) vs 2002 (inner circle) Retail Sales by Category')
plt.show()

In [ ]:
horeca = pd.read_excel('../data/Restaurant Clean.xlsx')
horeca.head()

In [ ]:
horeca = clean_year(horeca)
plot_bar(horeca, "HK Retail Sales - HK $",'Total Horeca Receipts')

In [ ]:
'Focus on the 2002 to 2005 period for a closer look'
'Create a date column by concatenating Year and Month'

horeca['Date'] = horeca['Year'].astype(str) + 'Q' + horeca['Quarter'].astype(str)
horeca_02_to_05 = horeca[horeca['Year'].isin(['2002','2003','2004','2005'])]

In [ ]:
plot_line(horeca_02_to_05, 'Total Horeca Receipts', "Sales of Restaurants in Hong Kong ('000 # Receipts) - 2002-2005")

In [ ]:
def plot_indexation_quarter(df, col_name, yaxis_label, title):

    df_2002 = df[df['Year'].isin(['2002'])]
    df_2003 = df[df['Year'].isin(['2003'])]
    df_2004 = df[df['Year'].isin(['2004'])]

    index_03vs02 = [int(tre/due*100) for tre,due in zip(df_2003[col_name],df_2002[col_name])]
    index_04vs02 = [int(quattro/due*100) for quattro,due in zip(df_2004[col_name],df_2002[col_name])]

    Quarter = ['Q1', 'Q2', 'Q3', 'Q4']
    Flat = [100, 100, 100, 100]
    data = {'Quarter': Quarter, 'Flat': Flat, '03 vs 02': index_03vs02, '04 vs 02': index_04vs02}
    df = pd.DataFrame (data)

    fig, ax = plt.subplots()

    ax.plot(df['Quarter'],df["03 vs 02"], label='03 vs 02', color="red")
    ax.plot(df['Quarter'],df["04 vs 02"], label='04 vs 02', color="green")
    ax.plot(df['Quarter'],df["Flat"], label='Flat', color="black", linewidth=0.25)


    ax.legend(loc=4) # upper left corner
    ax.set_xlabel('Quarter', fontsize=18)
    ax.set_ylabel(yaxis_label, fontsize=14)
    ax.set_title(title);

    return plt.show()

In [ ]:
plot_indexation_quarter(horeca, 'Total Horeca Receipts', 'Index Month over Month 2002', 'Restaurant Sales Index Month over Month 2002')

In [ ]:
'Focus on the 2002 and 2003 years to make a comparison'

horeca_02_and_03 = horeca[horeca['Year'].isin(['2002','2003'])]
horeca_02_and_03_not_all = horeca_02_and_03.drop(columns=['Total Horeca Receipts'])
sum_year__02_and_03_not_all = horeca_02_and_03_not_all[['Chinese restaurants',
                                                 'Non-Chinese restaurants',
                                                 'Fast food shops',
                                                 'Bars',
                                                 'Miscellaneous eating and drinking places','Year']].groupby(['Year']).agg("sum")


In [ ]:
fig, ax = plt.subplots(figsize = (24,12))

size = 0.3

cmap = plt.get_cmap("tab20c")
outer_colors = sns.color_palette("GnBu", 5)
inner_colors = sns.color_palette("GnBu", 5)

ax.pie(sum_year__02_and_03_not_all.iloc[1], 
       radius=1,
       labels=sum_year__02_and_03_not_all.columns, 
       wedgeprops=dict(width=size, edgecolor='w'),
       colors=outer_colors,
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.pie(sum_year__02_and_03_not_all.iloc[0], 
       radius=1-size, 
       colors=inner_colors,
       wedgeprops=dict(width=size, edgecolor='w'),
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.set(aspect="equal", title='2003 (outer circle) vs 2002 (inner circle) Retail Sales by Category')
plt.show()

In [ ]:
'Focus on the 2003 and 2004 years to make a comparison'

horeca_03_and_04 = horeca[horeca['Year'].isin(['2003','2004'])]
horeca_03_and_04_not_all = horeca_03_and_04.drop(columns=['Total Horeca Receipts'])
sum_year__03_and_04_not_all = horeca_03_and_04_not_all[['Chinese restaurants',
                                                 'Non-Chinese restaurants',
                                                 'Fast food shops',
                                                 'Bars',
                                                 'Miscellaneous eating and drinking places','Year']].groupby(['Year']).agg("sum")


In [ ]:
fig, ax = plt.subplots(figsize = (24,12))

size = 0.3

cmap = plt.get_cmap("tab20c")
outer_colors = sns.color_palette("GnBu", 5)
inner_colors = sns.color_palette("GnBu", 5)

ax.pie(sum_year__03_and_04_not_all.iloc[1], 
       radius=1,
       labels=sum_year__03_and_04_not_all.columns, 
       wedgeprops=dict(width=size, edgecolor='w'),
       colors=outer_colors,
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.pie(sum_year__03_and_04_not_all.iloc[0], 
       radius=1-size, 
       colors=inner_colors,
       wedgeprops=dict(width=size, edgecolor='w'),
       autopct='%1.1f%%',
       pctdistance=0.9)

ax.set(aspect="equal", title='2004 (outer circle) vs 2002 (inner circle) Retail Sales by Category')
plt.show()

In [ ]:
house_sales = pd.read_excel('../data/HK House Prices Clean.xlsx')
house_sales.head()

In [ ]:
house_sales = clean_year(house_sales)
plot_bar(house_sales, "HK House Price for Sales",'Average All Prices')

In [ ]:
'Focus on the 2002 to 2005 period for a closer look'
'Create a date column by concatenating Year and Month'

house_sales['Date'] = house_sales['Year'].astype(str) + 'Q' + house_sales['Quarter'].astype(str)
house_sales_02_to_05 = house_sales[house_sales['Year'].isin(['2002','2003','2004','2005'])]

In [ ]:
plot_line(house_sales_02_to_05, 'Average All Prices', "Sales of House in Hong Kong (# Sales) - 2002-2005")

In [ ]:
house_rent = pd.read_excel('../data/HK Rent Prices Clean.xlsx')
house_rent.head()

In [ ]:
house_rent = clean_year(house_rent)
plot_bar(house_rent, "HK House Price for Rent",'Average Rent Prices')

In [ ]:
'Focus on the 2002 to 2005 period for a closer look'
'Create a date column by concatenating Year and Month'

house_rent['Date'] = house_rent['Year'].astype(str) + 'Q' + house_rent['Quarter'].astype(str)
house_rent_02_to_05 = house_rent[house_rent['Year'].isin(['2002','2003','2004','2005'])]

In [ ]:
plot_line(house_rent_02_to_05, 'Average Rent Prices', "Rent Price of House in Hong Kong - 2002-2005")

In [ ]:
'Let us look at all info together in one dataframe'

In [ ]:
'Upload Retail Sales by Quarter'
retail_quarter = pd.read_excel('../data/Retail Clean Quarter.xlsx')
retail_quarter.head()

In [ ]:
retail_quarter['Date'] = retail_quarter['Year'].astype(str) + 'Q' + retail_quarter['Quarter'].astype(str)
retail_02_to_19 = retail_quarter[retail_quarter['Year'].isin(['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'])]

In [ ]:
horeca_02_to_19 = horeca[horeca['Year'].isin(['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'])]

In [ ]:
house_sales_02_to_19 = house_sales[house_sales['Year'].isin(['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'])]

In [ ]:
house_rent_02_to_19 = house_rent[house_rent['Year'].isin(['2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013','2014','2015','2016','2017','2018','2019'])]

In [ ]:
mainframe = retail_02_to_19.merge(horeca_02_to_19, on='Date',suffixes=('', '_y'))
mainframe=mainframe.drop(columns=['Year_y'])
mainframe=mainframe.drop(columns=['Quarter_y'])

In [ ]:
mainframe = mainframe.merge(house_sales_02_to_19, on='Date',suffixes=('', '_y'))
mainframe=mainframe.drop(columns=['Year_y'])
mainframe=mainframe.drop(columns=['Quarter_y'])

In [ ]:
mainframe = mainframe.merge(house_rent_02_to_19, on='Date',suffixes=('', '_rent'))
mainframe=mainframe.drop(columns=['Year_rent'])
mainframe=mainframe.drop(columns=['Quarter_rent'])

In [ ]:
def indexing(df, quarter_start, quarter_end):
    
    df['range'] = df['Date'].between(quarter_start, quarter_end, inclusive = True)
    lmt_df = df[(df['range']==True)]
    data = {'Quarter': lmt_df['Date']}
    
    index_frame = pd.DataFrame(data)
    col_names = [col for col in lmt_df.columns]
    col_names.remove('Year')
    col_names.remove('Quarter')
    col_names.remove('Date')
    col_names

    for elem in col_names:
        index_frame[elem] = [int(num/lmt_df[elem].iloc[0]*100) for num in lmt_df[elem]]

    return index_frame

In [ ]:
index_frame = indexing(mainframe, '2002Q2', '2010Q4')

In [ ]:
sel_cat = [name for name in index_frame.columns]
sel_cat.remove('Quarter')

In [ ]:
@interact
def Selection(Selection1=sel_cat, Selection2=sel_cat, Selection3=sel_cat, Selection4=sel_cat):
    
    fig, ax = plt.subplots(figsize=(20, 10))
    ax.plot(index_frame['Quarter'],index_frame[Selection1], label=Selection1,color="green")
    ax.plot(index_frame['Quarter'],index_frame[Selection2], label=Selection2,color="blue")
    ax.plot(index_frame['Quarter'],index_frame[Selection3], label=Selection3,color="black")
    ax.plot(index_frame['Quarter'],index_frame[Selection4], label=Selection4,color="red")
    
    plt.xticks(
        rotation=45, 
        horizontalalignment='right',
        fontweight='light',
        fontsize='small'  
    )
    ax.legend(loc=2)
    ax.set_xlabel('Quarter', fontsize=18)
    ax.set_ylabel("Index vs Jan 2002", fontsize=14)

In [ ]:
@interact
def Selection(Selection1=sel_cat, Selection2=sel_cat):
    
    cmap = sns.cubehelix_palette(dark=.3, light=.8, as_cmap=True)
    sizes=[100,200,300,400]
    fig, ax = plt.subplots(figsize=(20, 10))
    sns.scatterplot(data=mainframe, x=Selection1,y=Selection2, hue = 'Year', size = 'Quarter', palette="Set2", sizes=sizes) 
    